In [1]:
import requests

# Datos de la API
base_url = "http://development.localhost:8000"
api_key = "c01c600e6221df2"
api_secret = "3dccefad4336e43"

# Headers para autenticación
headers = {
    "Authorization": f"token {api_key}:{api_secret}"
}

# Endpoint que deseas consumir (ejemplo: listar documentos de la Doctype 'Item')
endpoint = "/api/method/frappe.auth.get_logged_user"

# Hacer la petición GET
response = requests.get(f"{base_url}{endpoint}", headers=headers)

# Ver el resultado
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code} - {response.text}")

{'message': 'Administrator'}


In [2]:
def link_transfer(name, doctype, links: dict):
    data = {
        f"customer_primary_{doctype.lower()}": name,  # ID del contacto como primario
    }

    for link in links:
        endpoint = f"/api/resource/{link['link_doctype']}/{link['link_name']}"
        response = requests.put(f"{base_url}{endpoint}", json=data, headers=headers)
    
    endpoint_link = f"/api/resource/{doctype}/{name}"
    data = {"links": links}
    response_link = requests.put(f"{base_url}{endpoint_link}", json=data, headers=headers)
    if response_link.status_code == 200 and response.status_code == 200:
        print(f"{link['link_name']} actualizado exitosamente")

In [3]:
import json

def create(doctype: str, data: dict, links: dict = None) -> str:
    endpoint = f'/api/resource/{doctype}'

    response = requests.post(f"{base_url}{endpoint}", json=data, headers=headers)

    name = None
    
    if links: data['links'] = links

    if response.status_code == 200:
        print(f"{doctype} creado exitosamente:")
        resp = response.json()
        print(resp)
        name = resp['data']['name']
        if links: link_transfer(name, doctype, data['links'])
    else:
        print(f"Error al crear el {doctype}: {response.status_code} - {response.text}")
        
    
    return json.loads(response.text)['data']

In [4]:
from api.sync import Sync

client = Sync('http://192.168.99.102/vtigercrm_2022', 'superadmin', 'MFaeyxCMTmRrUZiE')

service_id = "24x3"
product_id = "14x11"
assigned_user_id = "19x1"

19x113


In [5]:
module = "SalesOrder"
condition = {
    "cf_2059" :"2025-01-01"
}

module_object = client.search(module, condition)

In [7]:
def create_dependent_data(salesorder_data):
    mapping_cf = [
        {
            "type": "spouse",
            "firstname": "cf_2347",
            "middlename": "cf_2349",
            "lastname": "cf_2351",
            "ssn": "cf_2357",
            "dob": "cf_2355",
            "gender": "cf_2359",
            "apply": "cf_2389",
            "smoke": "cf_2709",
            "pregnant": "cf_2383",
            "relationship": "",
            "document": "cf_2603",
        },
        {
            "type": "dependent_1",
            "firstname": "cf_2405",
            "middlename": "cf_2407",
            "lastname": "cf_2409",
            "ssn": "cf_2417",
            "dob": "cf_2413",
            "gender": "cf_2411",
            "apply": "cf_2401",
            "smoke": "cf_2711",
            "pregnant": "cf_2423",
            "relationship": "cf_2431",
            "document": "cf_2605"
        },
        {
            "type": "dependent_2",
            "firstname": "cf_2443",
            "middlename": "cf_2445",
            "lastname": "cf_2447",
            "ssn": "cf_2455",
            "dob": "cf_2451",
            "gender": "cf_2449",
            "apply": "cf_2439",
            "smoke": "cf_2713",
            "pregnant": "cf_2459",
            "relationship": "cf_2465",
            "document": "cf_2607"
        },
        {
            "type": "dependent_3",
            "firstname": "cf_2479",
            "middlename": "cf_2481",
            "lastname": "cf_2483",
            "ssn": "cf_2491",
            "dob": "cf_2487",
            "gender": "cf_2485",
            "apply": "cf_2475",
            "smoke": "cf_2715",
            "pregnant": "cf_2495",
            "relationship": "cf_2501",
            "document": "cf_2609"
        },
        {
            "type": "dependent_4",
            "firstname": "cf_2515",
            "middlename": "cf_2517",
            "lastname": "cf_2519",
            "ssn": "cf_2527",
            "dob": "cf_2523",
            "gender": "cf_2521",
            "apply": "cf_2511",
            "smoke": "cf_2717",
            "pregnant": "cf_2531",
            "relationship": "cf_2537",
            "document": "cf_2639"
        },
        {
            "type": "dependent_5",
            "firstname": "cf_2645",
            "middlename": "cf_2647",
            "lastname": "cf_2649",
            "ssn": "cf_2687",
            "dob": "cf_2681",
            "gender": "cf_2679",
            "apply": "cf_2615",
            "smoke": "cf_2719",
            "pregnant": "cf_2685",
            "relationship": "cf_2699",
            "document": "cf_2683"
        }
    ]
    
    dependent_data = []
    
    documents = {
        'CIUDADANO': 'Passport',
        'ANTORCHA': 'Resident (I-551)',
        'RESIDENTE': 'Resident (I-551)',
        'ASILO POLITICO': 'Notice of Action (I-797)',
        'PASAPORTE DE VIAJERO': 'Refugee Travel (I-571)',
        'PERMISO DE TRABAJO': 'Employment Authorization (I-766)',
        'VISA': 'Visa (temporary I-551)',
        'NOTICIA DE ACCION': 'Notice of Action (I-797)',
        'I-94': 'Alien number or I-94 number',
    }
    
    relationships = {  
        'HIJO(A)': 'Child',
        'HIJASTRO(A)': 'Child-In-Law',
        'HERMANO(A)': 'Sibling',
        'PADRE/MADRE': 'Parent', 
        'TIO(A)': 'Pibling',
        'ABUELO(A)': 'Grandparent',
        'NIETO(A)': 'Grandchild',
        'SOBRINO(A)': 'Nibling', 
        'PRIMO(A)': 'Cousin',
        'SUEGRO(A)': 'Parent-In-Law',
    }
    
    for dependent_cf in mapping_cf:
        if salesorder_data[dependent_cf['apply']] != '':
            dependent_data.append(
                {
                    
                    "first_name": salesorder_data[dependent_cf['firstname']],
                    "middle_name": salesorder_data[dependent_cf['middlename']],
                    "last_name": salesorder_data[dependent_cf['lastname']],
                    "custom_ssn": salesorder_data[dependent_cf['ssn']],
                    "custom_day_of_birth": salesorder_data[dependent_cf['dob']],
                    "gender": salesorder_data[dependent_cf['gender']],
                    "custom_document": documents[salesorder_data[dependent_cf['document']]] if salesorder_data[dependent_cf['document']] != '' else '',
                    "apply": True if salesorder_data[dependent_cf['apply']] == "OBAMACARE" else False,
                    "smoke": False if salesorder_data[dependent_cf['smoke']] == "NOT" else True,
                    "pregnant": True if salesorder_data[dependent_cf['pregnant']] == "NOT" else False,
                    "relationship": 'Spouse' if dependent_cf['type'] == 'Spouse' else relationships[salesorder_data[dependent_cf['relationship']]],
                },
            )
    
    return dependent_data

In [8]:
def create_dependent(salesorder_data):
    dependent_data = create_dependent_data(salesorder_data)
    dependents = []
    for data in dependent_data:
        new_contact = create('Contact', data)
        other_info = {
            "relationship": data['relationship'],
            "smoke": data['smoke'],
            "apply": data['apply'],
            "pregnant": data['pregnant']
        }
        contact = {**new_contact, **other_info}
        dependents.append(contact)

    return dependents

In [9]:
import json
def get_id(name):
    endpoint = f'/api/resource/Customer'
    
    filters = [
        ["name", "=", name]
    ]

    response = requests.get(f"{base_url}{endpoint}", params={'filters': json.dumps(filters)}, headers=headers)

    if response.status_code == 200:
        resp = response.json()
        if len(resp['data']) > 0:
            return False
        else:
            print(f"No existe nadie llamado: {name}")
            return True 
    else:
        print(f"Error al filtrar: {filters}")
        return False

In [10]:
def create_owner_data(salesorder_data):
    contact = client.get_contact_from_id(salesorder_data['contact_id'])
    
    owner_data = []
    
    if contact['middlename'] and len(contact['middlename']) > 1:
        name = f"{contact['firstname']} {contact['middlename']} {contact['lastname']}"
    else:
        name = f"{contact['firstname']} {contact['lastname']}"
        
    documents = {
        'CIUDADANO': 'Passport',
        'ANTORCHA': 'Resident (I-551)',
        'RESIDENTE': 'Resident (I-551)',
        'ASILO POLITICO': 'Notice of Action (I-797)',
        'PASAPORTE DE VIAJERO': 'Refugee Travel (I-571)',
        'PERMISO DE TRABAJO': 'Employment Authorization (I-766)',
        'VISA': 'Visa (temporary I-551)',
        'NOTICIA DE ACCION': 'Notice of Action (I-797)',
        'I-94': 'Alien number or I-94 number',
    }
    
    if get_id(name):
        owner_data = {
            "contact_data": {
                "first_name": contact['firstname'],
                "middle_name": contact['middlename'],
                "last_name": contact['lastname'],
                "custom_day_of_birth": contact['dob'],
                "custom_ssn": ''.join([char for char in contact['ssn'] if char.isdigit()]),
                "gender": str(contact['gender']).lower().capitalize(),
                "email_ids": [
                    {
                        "email_id": contact['email'],  
                        "is_primary": 1,
                    }
                ],
                "phone_nos": [
                    {
                        "phone": contact['phone_1'],
                        "is_primary": 1,
                    }
                ]
            },
            "customer_data": {
                "customer_name": name,
                "customer_type": "Individual",
                "territory": "All Territories"
            },
            "address_data": {
                "address_title": name,
                "address_line1": salesorder_data['cf_2737'],
                "city": salesorder_data['cf_2739'],
                "state": salesorder_data['cf_2765'],
                "pincode": salesorder_data['cf_2743'],
                "country": "United States",
            },
            "additional_data": {
                "custom_document": documents[contact['cf_757']] if contact['cf_757'] != '' else '',
                "apply": True if contact['cf_1989'] == "OBAMACARE" else False,
                "smoke": False if contact['cf_775'] == "NOT" else True,
                "pregnant": True if contact['cf_2103'] == "NOT" else False,
                "relationship": 'Owner',
            },
        }
        print("Retorno owner_data")
        return owner_data
    else: 
        print("Retorno contact")
        return contact

In [12]:
def create_owner(salesorder_data):    
    if owner_data := create_owner_data(salesorder_data):
        try:
            customer = create('Customer', owner_data['customer_data'])
            customer_links = [
                {
                    "link_doctype": "Customer",
                    "link_name": customer['name']
                }
            ]
            create('Address', owner_data['address_data'], customer_links)
            create('Contact', owner_data['contact_data'], customer_links)
            return {**owner_data['contact_data'], **owner_data['addtional_data']}
        except:
            return owner_data

In [13]:
def create_salesorder(customer: list, dependents: list):
    endpoint = f'/api/resource/Sales Order'
    
    if customer['middlename'] and len(customer['middlename']) > 1:
        name = f"{customer['firstname']} {customer['middlename']} {customer['lastname']}"
    else:
        name = f"{customer['firstname']} {customer['lastname']}"
    
    data = {
        "customer": name, 
        "delivery_date": "2025-01-01",
        "custom_dependents": [
            {
                "contact": name,
                "relationship": customer['relationship'],
                "smoke": customer['smoke'],
                "pregnant": customer['pregnant'],
                "apply": customer['apply'],
            }
        ] + [
            {
                "contact": dependent['name'],
                "relationship": dependent['relationship'],
                "smoke": dependent['smoke'],
                "pregnant": dependent['pregnant'],
                "apply": dependent['apply'],
            } 
            for dependent in dependents
        ],
        "items": [
            {
                "item_code": "BS",  # Reemplaza por el código del artículo
                "qty": 1,
                "rate": 100.0  # Precio del artículo
            }
        ],
        "currency": "USD",  # Moneda (puede cambiarse según la configuración)
        "selling_price_list": "Standard Selling",  # Lista de precios
    }

    response = requests.post(f"{base_url}{endpoint}", json=data, headers=headers)

    if response.status_code == 200:
        print("Sales Order creada exitosamente:", response.json())
    else:
        print("Error al crear la Sales Order:", response.status_code, response.text)

In [ ]:
def create_bank_card(salesorder_data: list):
    endpoint = f'/api/resource/Bank Account'
    
    if len(salesorder_data['cf_1465']) > 1:
        card_types = {
            "Débito": "Debit",
            "Crédito": "Credit",
        }
        
        data = {
            "card_number": salesorder_data['cf_1465'], 
            "card_type": card_types[salesorder_data['cf_2157']],
            "expiration": salesorder_data['cf_1473'],
            "cvc": salesorder_data['cf_1477'],
        }

        response = requests.post(f"{base_url}{endpoint}", json=data, headers=headers)

        if response.status_code == 200:
            print("Sales Order creada exitosamente:", response.json())
        else:
            print("Error al crear la Sales Order:", response.status_code, response.text)

In [14]:
if len(["salesorder_data['cf_1479']"]) > 1:
        banks = {
            "JPMorgan Chase Bank",
            "Bank Of America",
            "Wells Fargo Bank",
            "Citibank",
            "U.S. Bank",
            "PNC Bank",
            "Truist Bank",
            "Capital One",
            "Goldman Sachs Bank"
            "SOFI Bank",
            "TD Bank",
            "BMO Bank",
            "Citizens Bank",
            "First Citizens Bank",
            "M&T Bank",
            "Fifth Third Bank",
            "Huntington National Bank",
            "American Express National Bank",
            "Key Bank",
            "Ally Bank",
            "HSBC Bank USA",
            "Forbright Bank",
            "Barclays"
        }
        
        bank_types = {
            "Saving",
            "Current"
        }
        
        bank_subtypes = {
            "Personal",
            "Business"
        }
        
        data = {
            "card_number": salesorder_data['cf_1465'], 
            "card_type": card_types[salesorder_data['cf_2157']],
            "expiration": salesorder_data['cf_1473'],
            "cvc": salesorder_data['cf_1477'],
        }

In [12]:
salesorder_data = module_object[31]

In [13]:
owner = create_owner(salesorder_data)
owner

Retorno contact


{'firstname': 'LORENA',
 'middlename': '',
 'lastname': 'ESPINOZA',
 'ssn': '080-90-8761',
 'dob': '1976-07-15',
 'gender': 'FEMALE',
 'phone_1': '3476584981',
 'phone_2': '',
 'email': 'luji27@hotmail.com',
 'document': 'CIUDADANO',
 'apply': False,
 'smoke': False,
 'pregnant': False,
 'relationship': 'Owner'}

In [14]:
dependents = create_dependent(salesorder_data)

Contact creado exitosamente:
{'data': {'name': 'SANTINO BEDOYA RAMIREZ-6', 'owner': 'Administrator', 'creation': '2024-10-10 17:27:10.915556', 'modified': '2024-10-10 17:27:10.915556', 'modified_by': 'Administrator', 'docstatus': 0, 'idx': 0, 'first_name': 'SANTINO', 'middle_name': '', 'last_name': 'BEDOYA RAMIREZ', 'full_name': 'SANTINO BEDOYA RAMIREZ', 'email_id': '', 'sync_with_google_contacts': 0, 'custom_day_of_birth': '2017-09-28', 'custom_ssn': '', 'custom_document': '', 'status': 'Passive', 'gender': 'Male', 'phone': '', 'mobile_no': '', 'pulled_from_google_contacts': 0, 'is_primary_contact': 0, 'is_billing_contact': 0, 'unsubscribed': 0, 'doctype': 'Contact', 'links': [], 'email_ids': [], 'phone_nos': []}}
Contact creado exitosamente:
{'data': {'name': 'TRISTANO BEDOYA RAMIREZ-6', 'owner': 'Administrator', 'creation': '2024-10-10 17:27:11.285702', 'modified': '2024-10-10 17:27:11.285702', 'modified_by': 'Administrator', 'docstatus': 0, 'idx': 0, 'first_name': 'TRISTANO', 'midd

In [ ]:
#bank_account_name = create_bank_account(salesorder_data)

In [21]:
salesorder_name = create_salesorder(owner, dependents)

Sales Order creada exitosamente: {'data': {'name': 'SAL-ORD-2024-00010', 'owner': 'Administrator', 'creation': '2024-10-10 17:31:26.298374', 'modified': '2024-10-10 17:31:26.298374', 'modified_by': 'Administrator', 'docstatus': 0, 'idx': 0, 'title': '{customer_name}', 'naming_series': 'SAL-ORD-.YYYY.-', 'customer': 'LORENA ESPINOZA', 'customer_name': 'LORENA ESPINOZA', 'order_type': 'Sales', 'transaction_date': '2024-10-10', 'delivery_date': '2025-01-01', 'custom_renew': 0, 'custom_consent': '', 'custom_app_review': '', 'company': 'Mabe Center', 'skip_delivery_note': 0, 'currency': 'USD', 'conversion_rate': 1.0, 'selling_price_list': 'Standard Selling', 'price_list_currency': 'USD', 'plc_conversion_rate': 1.0, 'ignore_pricing_rule': 0, 'reserve_stock': 0, 'total_qty': 1.0, 'total_net_weight': 0.0, 'base_total': 100.0, 'base_net_total': 100.0, 'total': 100.0, 'net_total': 100.0, 'tax_category': '', 'exempt_from_sales_tax': 0, 'base_total_taxes_and_charges': 0.0, 'total_taxes_and_charges